## Chemical Measurements Data and Population
This Notebook contains the Chemical Data Measurements and population by town visualization and the health metric by Population Visualization
### Section 1: Data

In [1]:
import pandas as pd
import numpy as np
import altair as alt
import IPython
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

alt.renderers.enable('default')

alt.data_transformers.enable('json')


pd.set_option('display.max_columns', None)

In [2]:
chem_data = pd.read_csv('assets/chem_data_merged.csv', index_col=0)
health_df = pd.read_csv('assets/health_metric.csv', index_col=0)
print('The size of the chem_data dataframe')
print('columns:', chem_data.shape[1])
print('rows   :', chem_data.shape[0])
display(chem_data.sample(5))
print('The size of the health metric dataframe')
print('columns:', health_df.shape[1])
print('rows   :', health_df.shape[0])
display(health_df.sample(5))



The size of the chem_data dataframe
columns: 26
rows   : 284535


,LakeID,LakeStationNo,LakeStationType,Lat,Long,Town,ProjectID,VisitDate,VisitNumber,StartTime,CollectionMethodID,Depth,ActivityCategory,CharacteristicID,Symbol,Result,Calcs,ProjRemark,RemarkCode,DepthStratumCode,year,population,CharacteristicName,UnitCode,SampleFraction,NormResult
54510,COW MOUNTAIN,1,Pelagic,44.56140,-71.70250,GRANBY,AcidLake,1988-05-05,1,1005.0,PlasticKemm,1.0,R1,DCL,NaN,0.28,Y,NaN,NaN,NaN,1988,82.0,Dissolved Chloride,mg/l,Dissolved,0.002648
188691,PARKER,1,Pelagic,44.71926,-72.23425,GLOVER,LayMonQC,2010-06-28,1,1245.0,Hose,6.0,R1,CHLA,NaN,8.80,Y,NaN,NaN,NaN,2010,1122.0,Chlorophyll-a,ug/l,NaN,0.035315
136999,KNAPP BROOK #1,1,Pelagic,43.44637,-72.56222,CAVENDISH,Laymon,2020-07-06,1,1810.0,Secchi,NaN,Reg,SECCHI,NaN,1.20,Y,NaN,NaN,NaN,2020,NaN,Secchi transparency,m,NaN,0.075364
281145,WINONA,1,Pelagic,44.17527,-73.09047,BRISTOL,SpringTP,1996-04-24,1,1200.0,Kemmerer,1.0,Reg,REGALK,NaN,123.50,Y,NaN,NaN,NaN,1996,3820.0,Alkalinity,mg/l,NaN,0.656732
8464,BEEBE (HUBDTN),1,Pelagic,43.73440,-73.18330,HUBBARDTON,LakeAsmt,2003-08-13,1,1300.0,Hydrolab,12.0,Reg,COND,NaN,262.00,Y,NaN,NaN,NaN,2003,734.0,Conductivity,umho/cm,NaN,0.024833


The size of the health metric dataframe
columns: 2
rows   : 90


,Lake,Health_Score
50,RAPONDA,1.075355
25,SUNSET (BRKFLD),0.953818
52,ARROWHEAD MOUNTAIN,4.387066
75,MAIDSTONE,5.027080
24,NICHOLS,1.725785


### Section 2: Clean For Visualization

In [3]:
chem_data['VisitDate'] = pd.to_datetime(chem_data['VisitDate'])
chem_data['year'] = chem_data['VisitDate'].dt.year
chem_data.dropna(subset=['population'], inplace=True)
chem_data = chem_data[chem_data['population'] != 0]

### Section 3: Chemical Metrics and Population by town Visualization

In [4]:

def show_measurements(df):
    df_for_measurements = df[['year', 'Town', 'CharacteristicID', 'CharacteristicName', 'UnitCode', 'Result', 'NormResult']].groupby(['Town', 'year', 'CharacteristicID', 'CharacteristicName', 'UnitCode']).mean().reset_index()
    df_for_count = df[['Town', 'CharacteristicID', 'CharacteristicName', 'UnitCode', 'Result']].groupby(['Town', 'CharacteristicID', 'CharacteristicName', 'UnitCode']).count().reset_index()
    df_for_population = df[['Town', 'year', 'population']].copy().drop_duplicates()
    
    

    options = list(chem_data.groupby('Town')['Town'].count().sort_values(ascending=False).index)
    town_dropdown = alt.binding_select(options=options, name='Towns')
    selection_d = alt.selection_single(fields=['Town'], init={'Town': options[0]}, bind=town_dropdown)


    
    selection = alt.selection_multi(fields=['CharacteristicID'], clear=False)
    opacity_selection = alt.condition(selection, alt.value(1), alt.value(.2))

    total_measurements = alt.Chart(df_for_measurements).mark_line().encode(
        x=alt.X('year:O'),
        y=alt.Y('mean(NormResult):Q'),
        color=alt.Color('CharacteristicID:N', legend=None)
    )
    point_measure = total_measurements.mark_circle().encode(
        x=alt.X('year:O'),
        y=alt.Y('mean(NormResult):Q'),
        color=alt.Color('CharacteristicID:N', legend=None),
        tooltip=[alt.Tooltip('CharacteristicID'), alt.Tooltip('mean(NormResult)')]
    ).interactive()

    measures = (total_measurements + point_measure).add_selection(
        selection_d
    ).transform_filter(
        selection
    ).transform_filter(selection_d).properties(
        width=700,
        height=400
    )

    total_counts = alt.Chart(df_for_count).mark_bar().encode(
        y=alt.Y('CharacteristicID:N', sort='-x', axis=alt.Axis(title=None)),
        x=alt.X('sum(Result):Q'),
        tooltip=[alt.Tooltip('CharacteristicName'), alt.Tooltip('UnitCode'), alt.Tooltip('sum(Result)', title='Total Measurements')],
        opacity=opacity_selection,
        color=alt.Color('CharacteristicID:N')
    )

    text = total_counts.mark_text(dx=20).encode(
        text='sum(Result):Q',
        opacity=opacity_selection
    )

    pop = alt.Chart(df_for_population).mark_line().encode(
        x=alt.X('year:O'),
        y=alt.Y('population:Q')
    ).interactive()

    pop_point = alt.Chart(df_for_population).mark_circle().encode(
        x=alt.X('year:O'),
        y=alt.Y('population:Q'),
        tooltip=alt.Tooltip('population')
    ).interactive()
    pop_charts = (pop + pop_point).add_selection(
        selection_d
    ).transform_filter(
        selection_d
    ).properties(
        width=700
    )
    counts_and_text = (total_counts + text).add_selection(
        selection_d, 
        selection
    ).transform_filter(
        selection_d
    ).properties(
        width=150
    )

    return ((measures & pop_charts).resolve_scale(x='shared') | counts_and_text).properties(
        title=alt.TitleParams(text='Lake Health Measures and Population',
                            subtitle='Is there more testing and are the values higher in larger population areas')
    ).configure(padding={"left": 5, "top": 5, "right": 100, "bottom": 200})
show_measurements(chem_data)

alt.HConcatChart(...)

### Section 4: Health Metric and 2019 population data merging

In [5]:
# now to explore the health metric and population of towns
health_and_pop = health_df.merge(chem_data[['LakeID', 'Town', 'year', 'population']], how='left', left_on='Lake', right_on='LakeID').drop('LakeID', axis=1)
health_and_pop

,Lake,Health_Score,Town,year,population
0,LYFORD,1.873349,WALDEN,1990.0,703.0
1,LYFORD,1.873349,WALDEN,1990.0,703.0
2,LYFORD,1.873349,WALDEN,1997.0,762.0
3,LYFORD,1.873349,WALDEN,1997.0,762.0
4,LYFORD,1.873349,WALDEN,1997.0,762.0
...,...,...,...,...,...
149678,ISLAND,1.047180,BRIGHTON,2018.0,1188.0
149679,ISLAND,1.047180,BRIGHTON,2018.0,1188.0
149680,ISLAND,1.047180,BRIGHTON,2018.0,1188.0
149681,ISLAND,1.047180,BRIGHTON,2018.0,1188.0


In [6]:
pop_to_use = health_and_pop[health_and_pop['year'] == 2019]
grouped_health = pop_to_use[['Town', 'population', 'Health_Score']].groupby('Town').mean().reset_index().sort_values(by='population', ascending=False)
# grouped_health['Health_Score'] = np.log(grouped_health['Health_Score'])
corr = grouped_health.corr().iloc[0, 1]

### Section 5: Health Metric and 2019 population scatterplot

In [7]:
base = alt.Chart(grouped_health).mark_circle().encode(
    x=alt.X('population:Q'),
    y=alt.Y('Health_Score:Q'),
    tooltip=alt.Tooltip('Town')
).properties(
    title=alt.TitleParams('Town population and Health Score',
                    subtitle=f'Correlation of {round(corr, 5)}')
)

base

alt.Chart(...)

## Documentation

In [8]:
%load_ext watermark
%watermark --iversions

altair : 4.1.0
IPython: 7.29.0
numpy  : 1.21.5
pandas : 1.3.4
sys    : 3.9.9 | packaged by conda-forge | (main, Dec 20 2021, 02:36:06) [MSC v.1929 64 bit (AMD64)]

